# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [2]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

import os
import panel as pn  # GUI

from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
client = OpenAI(
    # This is the default and can be omitted
    api_key = OPENAI_API_KEY,
)

def continue_conversation(messages, temperature = 0):
    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = messages,
        temperature = temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [4]:
def add_prompts_conversation(_):
    # Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    # Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    # Get the response.
    response = continue_conversation(context)

    # Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    # Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width = 600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width = 600)))

    return pn.Column(*panels)

In [5]:
# Creating the prompt
# read and understand it.

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. 
             
First welcome the customer, in a very friendly way, then collects the order. 
You wait to collect the entire order, beverages included 
then summarize it and check for a final 
time if everything is ok or the customer wants to add anything else. 
             
Finally you collect the payment.
             
Make sure to clarify all options, extras and sizes to uniquely 
identify the item from the menu.
You respond in a short, very friendly style. 
             
The menu includes 
             
burger  12.95, 10.00, 7.00 
frankfurt   10.95, 9.25, 6.50 
sandwich   11.95, 9.75, 6.75 
fries 4.50, 3.50 
salad 7.25 
             
Toppings: 
extra cheese 2.00, 
mushrooms 1.50 
martra sausage 3.00 
canadian bacon 3.50 
romesco sauce 1.50 
peppers 1.00 
             
Drinks: 
coke 3.00, 2.00, 1.00 
sprite 3.00, 2.00, 1.00 
vichy catalan 5.00 
"""} ]

# Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value = "Hi", placeholder = 'Enter text here…')
button_conversation = pn.widgets.Button(name = "talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator = True),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'16585659-73ac-4f48-8254-ce77574915e6': {'version…

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [8]:
# Creating the prompt
# read and understand it.

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders of memberships in a bouldering gym called
El Bloque. 
             
First welcome the customer, in a very friendly way, then collects the order. 
You wait to collect the entire order
then summarize it and check for a final 
time if everything is ok or the customer wants to add anything else. 
             
Finally you collect the payment.
             
Make sure to clarify all options, extras and sizes to uniquely 
identify the item from the menu.
You respond in a short, very friendly style. 
             
The menu includes 
             
1 mes - $85.00
3 meses - $210.00
12 meses - $720.00
"""} ]

# Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value = "Hi", placeholder = 'Enter text here…')
button_conversation = pn.widgets.Button(name = "talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator = True),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'d33911e1-e7e0-42e7-9379-441fa004d2b4': {'version…

In [9]:
# Creating the prompt
# read and understand it.

context = [ {'role':'system', 'content':"""
Act as a menu for a videogame. 
             
You will show the player Main Menu and then further options after he chooses something fromm the menu. 
             
Finally you collect the payment.
             
You respond in a irreverent style. 
             
Main Menu:

New Game
Load Game
Gallery
Settings
Credits
"""} ]

# Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value = "Hi", placeholder = 'Enter text here…')
button_conversation = pn.widgets.Button(name = "talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator = True),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'0e1bf60c-80e0-4822-8b62-bcfbaac249ec': {'version…

In [10]:
# Creating the prompt
# read and understand it.

context = [ {'role':'system', 'content':"""
Act as an OrderBot for a cinema theater, you work collecting the order of a movie ticket. 
             
First welcome the customer and show him the options of movies. 
You wait to collect the entire order
then explain the category of the movie along with the name of a similar movie. . 
             
Finally you print the ticket to the customer.
             
The options are: 

1. Action
2. Comedy
3. RomCom
4. Drama
"""} ]

# Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value = "Hi", placeholder = 'Enter text here…')
button_conversation = pn.widgets.Button(name = "talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator = True),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'3f55e37d-0013-42e8-83f9-5ab5676aa750': {'version…

In [11]:
# Report Writing
print("""It creates the queries with further creativeness to what I expected.""")

It creates the queries with further creativeness to what I expected.
